In [1]:
using Turing
using Random
using ForwardDiff
using LimberJack
using CSV
using NPZ
using FITSIO
using Plots
using LinearAlgebra
using BlockArrays
using Distributed
using DataFrames

In [2]:
# Add four processes to use for sampling.
#addprocs(4)

# Initialize everything on all the processes.
# Note: Make sure to do this after you've already loaded Turing,
#       so each process does not have to precompile.
#       Parallel sampling may fail silently if you do not do this.
#@everywhere using Turing, LimberJack

In [7]:
desgs_nzs = FITS("/home/jaime/PhD/LimberJack.jl/Notebooks/data/y1_redshift_distributions_v1.fits")
desgs_nz = read(desgs_nzs["nz_source_mcal"], "BIN2")
desgs_zs = read(desgs_nzs["nz_source_mcal"], "Z_MID");
desgg_nzs = FITS("/home/jaime/PhD/LimberJack.jl/Notebooks/data/y1_redshift_distributions_v1.fits")
desgg_nz = read(desgg_nzs["nz_source_mcal"], "BIN2")
desgg_zs = read(desgg_nzs["nz_source_mcal"], "Z_MID");
ell = npzread("/home/jaime/PhD/LimberJack.jl/Notebooks/data/cl_DESgc__2_DESwl__3.npz")["ell"]
ell = [Int(floor(l)) for l in ell];

In [8]:
path = "/home/jaime/PhD/LimberJack.jl/Notebooks/data/"
datas = [Data("DESgc", "DESgc", 2 , 2, path=path),
         Data("DESgc", "DESwl", 2 , 3, path=path)];
Cls_metas = Cls_meta(datas, path=path);
cov_tot = Cls_metas.cov_tot;
data_vector = Cls_metas.data_vector;

In [9]:
@model function model(data)
    Ωm ~ Uniform(0.2, 0.3)
    h ~ Uniform(0.6, 0.8)
    s8 ~ Uniform(0.7, 1.0)
    cosmology = LimberJack.Cosmology(Ωm, 0.05, h, 0.96, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    tg = NumberCountsTracer(cosmology, desgg_zs, desgg_nz, 2.)
    ts = WeakLensingTracer(cosmology, desgs_zs, desgs_nz)
    gg = [angularCℓ(cosmology, tg, tg, ℓ) for ℓ in ell]
    gs = [angularCℓ(cosmology, tg, tg, ℓ) for ℓ in ell]
    predictions = [gg, gs]
    predictions = vcat(predictions...)
    data ~ MvNormal(predictions, cov_tot)
end;

iterations = 50
step_size = 0.005
samples_per_step = 10
cores = 4

# Start sampling.

folname = string("DES_cobmined_test_", "stpsz_", step_size, "_smpls_", samples_per_step)
if isdir(folname)
    println("Folder already exists")
    if isfile(joinpath(folname, "chain.jls"))
        println("Restarting from past chain")
        past_chain = read(joinpath(folname, "chain.jls"), Chains)
        new_chain = sample(model(data_vector), HMC(step_size, samples_per_step), iterations,
                           progress=true; save_state=true, resume_from=past_chain)
    end
else
    mkdir(folname)
    println("Created new folder")
    new_chain = sample(model(data_vector), HMC(step_size, samples_per_step),
                iterations, progress=true; save_state=true)
end

info = describe(new_chain)[1]
fname_info = string("info.csv")
CSV.write(joinpath(folname, fname_info), info)


fname_jls = string("chain.jls")
write(joinpath(folname, fname_jls), new_chain)
    
fname_csv = string("chain.csv")
CSV.write(joinpath(folname, fname_csv), new_chain)

Sampling: 100%|█████████████████████████████████████████| Time: 0:06:26


Created new folder


"DES_cobmined_test_stpsz_0.005_smpls_10/chain.csv"